In [1]:
#Systematic Methods of Chemical Process Design
# 16.2 SEQUENTIAL SYNTHESIS
# 16.2.3 Prediction of Matches for Minimizing the Number of Units
# 16.2.4 Automatic Derivation of Network Structures

import numpy as np
import pandas as pd
import pyomo.environ as pyo

In [2]:
# Stream data for example 16.4
# Last stream of the hot and cold streams are utility streams
hot_streams = pd.DataFrame(data=[[1,400,120],[2,340,120],
                                 # utility stream
                                 [60,501,500]],
                       index=[1,2,3],
                       columns=['Fcp','Tin', 'Tout'])

hot_streams['h'] = 1

cold_streams = pd.DataFrame(data=[[1.5,160,400],[1.3,100,250], 
                                  #utility stream
                                  [22.5,20,30]],
                       index=[1,2,3],
                       columns=['Fcp','Tin', 'Tout'])
cold_streams['h'] = 1

tmapp = 20

In [19]:
# # Last stream of the hot and cold streams are utility streams
# hot_streams = pd.DataFrame(data=[[0.15,250,40],[0.25,200,80],
#                                  # utility stream
#                                  [7.5,501,500]],
#                        index=[1,2,3],
#                        columns=['Fcp','Tin', 'Tout'])

# hot_streams['h'] = 1

# cold_streams = pd.DataFrame(data=[[0.2,20,180],[0.3,140,230], 
#                                   #utility stream
#                                   [1.0,20,30]],
#                        index=[1,2,3],
#                        columns=['Fcp','Tin', 'Tout'])
# cold_streams['h'] = 1

# tmapp = 10

In [3]:
#Temperature Intervals
T_hot  = np.unique(hot_streams[['Tin','Tout']])
T_cold = np.unique(cold_streams[['Tin','Tout']])

T_intervals = pd.Series(data = np.unique(np.concatenate((T_hot,T_cold)))[::-1])
T_intervals.index += 1

#Shifted Temperatures
T_shifted = pd.Series(data = np.unique(np.concatenate((T_hot-tmapp/2,T_cold+tmapp/2)))[::-1])
T_shifted.index += 1

In [4]:
#Heat Load per Interval
i_hot = pd.Series(data = ((hot_streams['Tin']-tmapp/2 >= T) & (hot_streams['Tout']-tmapp/2 <= Tnext) for T,Tnext in zip(T_shifted,T_shifted[1:])))
i_hot.index += 1

i_cold = pd.Series(data = ((cold_streams['Tout']+tmapp/2 >= T) & (cold_streams['Tin']+tmapp/2 <= Tnext) for T,Tnext in zip(T_shifted,T_shifted[1:])))
i_cold.index += 1

In [5]:
model = pyo.ConcreteModel()

# Sets
model.k  = pyo.RangeSet(0,len(T_shifted))
model.st = pyo.RangeSet(1,len(T_shifted)-1)
model.i  = pyo.RangeSet(1,len(hot_streams)) 
model.j  = pyo.RangeSet(1,len(cold_streams))

# Variables
model.Q      = pyo.Var(model.i, model.j, model.st, domain=pyo.NonNegativeReals, initialize = 0)
model.r      = pyo.Var(model.i, model.k,domain=pyo.NonNegativeReals, initialize = 0)

# Binary Variables
model.y = pyo.Var(model.i, model.j, domain=pyo.Binary)

# Parameters - Upper bounds for heat exchange
def bounds(model,i,j):
    hot  = hot_streams['Fcp'][i]*(hot_streams['Tin'][i] - hot_streams['Tout'][i])
    cold = cold_streams['Fcp'][j]*(cold_streams['Tout'][j] - cold_streams['Tin'][j])
    return min(hot,cold)
    

model.U = pyo.Param(model.i, model.j, domain=pyo.NonNegativeReals, initialize = bounds)

# Constraints
# Total energy exchanged by hot stream i
def energy_hot(model,i,k):
    return model.r[i,k] - model.r[i,k-1] \
                          + sum(model.Q[i,j,k] for j in model.j)\
                          == hot_streams['Fcp'][i]*(T_shifted[k] - T_shifted[k+1])*int(i_hot[k][i])

model.cons_hot = pyo.Constraint(model.i, model.st, rule = energy_hot)

# Energy exchanged by cold stream j in stage k
def energy_cold(model,j,k):
    if i_cold[k][j]:
        return sum(model.Q[i,j,k] for i in model.i) == cold_streams['Fcp'][j]*(T_shifted[k] - T_shifted[k+1])
    return pyo.Constraint.Skip

model.cons_cold = pyo.Constraint(model.j,model.st, rule = energy_cold)

# First and last residual constraint
def res_0(model,i):
    return  model.r[i,0] == 0

def res_k(model,i):
    return  model.r[i,len(model.st)] == 0

model.first_res = pyo.Constraint(model.i,rule = res_0)
model.last_res = pyo.Constraint(model.i,rule = res_k)

# Constrained matches
# model.cmatch1 = pyo.Constraint(expr = sum(model.Q[3,5,k] for k in model.st) == 0)
# model.cmatch1 = pyo.Constraint(expr = sum(model.Q[3,7,k] for k in model.st) == 0)

# Minimum number of units above and below the pinch
def units(model,i,j):
    return sum(model.Q[i,j,k] for k in model.st) - model.U[i,j]*model.y[i,j] <= 0

model.cons_y = pyo.Constraint(model.i, model.j, rule = units)

# # Objective
# def obj_expression(model):
#     return sum(sum(hot_utility['Cost'][m]*model.Q_hot[m,k] for m in model.m) for k in model.st)\
#         + sum(sum(cold_utility['Cost'][n]*model.Q_cold[n,k] for n in model.n) for k in model.st)

model.OBJ = pyo.Objective(expr = pyo.summation(model.y), sense = pyo.minimize)
# Results
# results = pyo.SolverFactory("cbc", executable="C:\\Users\\Dell\\Downloads\\cbc-win64\\cbc").solve(model,tee=False)
results = pyo.SolverFactory("glpk").solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 5.0
  Upper bound: 5.0
  Number of objectives: 1
  Number of constraints: 50
  Number of variables: 121
  Number of nonzeros: 253
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.2333216667175293
# ----------------------------------------------------------
#   Solution Information
# -------------------------------------

In [6]:
# Minimum area cost
m2 = pyo.ConcreteModel()

# Sets
m2.i = pyo.Set(initialize = hot_streams.index)
m2.j = pyo.Set(initialize = cold_streams.index)

# Parameters 
m2.U = pyo.Param(m2.i,m2.j, initialize = 1, mutable=True)
# m2.U[1,1] = 1
# m2.U[1,2] = 0.5

# Initialization
# # Hot temperatures
def th_in_init(m2,i,j):
    if model.y[i,j].value==1:
        t = hot_streams['Tin'][i] 
    else:
        t = 0
    return t

def th_out_init(m2,i,j):
    if model.y[i,j].value==1:
        t = hot_streams['Tin'][i] - sum(model.Q[i,j,k] for k in model.st)/hot_streams['Fcp'][i] 
    else:
        t = 0
    return t

def th_bound(m2,i,j):
    if model.y[i,j].value==1:
        up = hot_streams['Tin'][i] 
        lb = hot_streams['Tout'][i] 
    else:
        up = 0
        lb = 0
    return (lb,up)

# Hot streams FCP
def fcp_hot(m2,i,j):
    if model.y[i,j].value==1:
        fcp = hot_streams['Fcp'][i]
    else:
        fcp = 0 
    return fcp

def fcp_hot_bound(m2,i,j):
    if model.y[i,j].value==1:
        up = hot_streams['Fcp'][i]
        lb = 0
    else:
        up = 0
        lb = 0
    return (lb,up)

def fcp_hot_split(m2,i,j,p):
    if model.y[i,j].value==1:
        fcp = hot_streams['Fcp'][i]
    else:
        fcp = 0 
    return fcp

def fcps_hot_bound(m2,i,j,p):
    if model.y[i,j].value==1:
        up = hot_streams['Fcp'][i]
        lb = 0
    else:
        up = 0
        lb = 0
    return (lb,up)

# # Cold temperatures
def tc_in_init(m2,i,j):
    if model.y[i,j].value==1:
        up = cold_streams['Tin'][j]
    else:
        up = 0
    return up

def tc_out_init(m2,i,j):
    if model.y[i,j].value==1:
        t = cold_streams['Tin'][j] + sum(model.Q[i,j,k] for k in model.st)/cold_streams['Fcp'][j]
    else:
        t = 0
    return t

def tc_bound(m2,i,j):
    if model.y[i,j].value==1:
        up = cold_streams['Tout'][j] 
        lb = cold_streams['Tin'][j] 
    else:
        up = 0
        lb = 0
    return (lb,up)

# Cold streams FCP
def fcp_cold(m2,i,j):
    if model.y[i,j].value==1:
        fcp = cold_streams['Fcp'][j]
    else:
        fcp = 0 
    return fcp

def fcp_cold_bound(m2,i,j):
    if model.y[i,j].value==1:
        up = cold_streams['Fcp'][j]
        lb = 0
    else:
        up = 0
        lb = 0
    return (lb,up)

def fcp_cold_split(m2,i,j,p):
    if model.y[i,j].value==1:
        fcp = cold_streams['Fcp'][j]
    else:
        fcp = 0 
    return fcp

def fcps_cold_bound(m2,i,j,p):
    if model.y[i,j].value==1:
        up = cold_streams['Fcp'][j]
        lb = 0
    else:
        up = 0
        lb = 0
    return (lb,up)

# Variables
# # flows out of initial splitter
m2.fs_c     = pyo.Var(m2.i,m2.j, domain=pyo.NonNegativeReals, initialize = 0, bounds = fcp_cold_bound)
m2.fs_h     = pyo.Var(m2.i,m2.j, domain=pyo.NonNegativeReals, initialize = 0, bounds = fcp_hot_bound)

# # flows to other exchangers and temperatures in and out of exchangers
# flow into heat exchanger and out (s) ~after~ exchanger splitter
m2.fhex_c   = pyo.Var(m2.i,m2.j, domain=pyo.NonNegativeReals, initialize = 0, bounds = fcp_cold_bound)
m2.fhexs_c  = pyo.Var(m2.i,m2.j,m2.i, initialize = 0, domain=pyo.NonNegativeReals)

m2.fhex_h   = pyo.Var(m2.i,m2.j, domain=pyo.NonNegativeReals, initialize = 0, bounds = fcp_hot_bound)
m2.fhexs_h  = pyo.Var(m2.i,m2.j,m2.j, initialize = 0, domain=pyo.NonNegativeReals)

#  temperature into and out of heat exchanger
m2.tc_in    = pyo.Var(m2.i,m2.j, domain=pyo.NonNegativeReals, initialize = tc_in_init, bounds = tc_bound)
m2.tc_out   = pyo.Var(m2.i,m2.j, domain=pyo.NonNegativeReals, initialize = tc_out_init, bounds = tc_bound)
m2.th_in    = pyo.Var(m2.i,m2.j, domain=pyo.NonNegativeReals, initialize = th_in_init, bounds = th_bound)
m2.th_out   = pyo.Var(m2.i,m2.j, domain=pyo.NonNegativeReals, initialize = th_out_init, bounds = th_bound)

# flows into initial mixer
m2.fmix_c    = pyo.Var(m2.i,m2.j, initialize = 0, domain=pyo.NonNegativeReals)
m2.fmix_h    = pyo.Var(m2.i,m2.j, initialize = 0, domain=pyo.NonNegativeReals)

# Equations
# Initial splitter
def init_spl_c(m2,j):
    return sum(m2.fs_c[i,j] for i in m2.i) == cold_streams['Fcp'][j]

m2.cons_spl_c = pyo.Constraint(m2.j, rule = init_spl_c)

def init_spl_h(m2,i):
    return sum(m2.fs_h[i,j] for j in m2.j) == hot_streams['Fcp'][i]

m2.cons_spl_h = pyo.Constraint(m2.i, rule = init_spl_h)

# Mixer at the entry of heat exchanger
def mix_hex_c(m2,i,j):
    return m2.fs_c[i,j] + sum(m2.fhexs_c[i,j,p] for p in m2.i if p!=i) == m2.fhex_c[i,j]

m2.cons_hexm_c = pyo.Constraint(m2.i,m2.j, rule = mix_hex_c)

def mix_hex_c_heat(m2,i,j):
    return m2.fs_c[i,j]*cold_streams['Tin'][j] + sum(m2.fhexs_c[i,j,p]*m2.tc_out[p,j] for p in m2.i if p!=i)\
        == m2.fhex_c[i,j]*m2.tc_in[i,j]

m2.cons_hexh_c = pyo.Constraint(m2.i,m2.j, rule = mix_hex_c_heat)

def mix_hex_h(m2,i,j):
    return m2.fs_h[i,j] + sum(m2.fhexs_h[i,j,p] for p in m2.j if p!=j) == m2.fhex_h[i,j]

m2.cons_hexm_h = pyo.Constraint(m2.i,m2.j, rule = mix_hex_h)

def mix_hex_h_heat(m2,i,j):
    return m2.fs_h[i,j]*hot_streams['Tin'][i] + sum(m2.fhexs_h[i,j,p]*m2.th_out[i,p] for p in m2.j if p!=j)\
        == m2.fhex_h[i,j]*m2.th_in[i,j]

m2.cons_hexh_h = pyo.Constraint(m2.i,m2.j, rule = mix_hex_h_heat)

# # Splitter at the outlet of the exchanger 
def spl_hex_c(m2,i,j):
    return m2.fhex_c[i,j] == sum(m2.fhexs_c[p,j,i] for p in m2.i if p!=i) + m2.fmix_c[i,j]

m2.cons_hexs_c = pyo.Constraint(m2.i,m2.j, rule = spl_hex_c)

def spl_hex_h(m2,i,j):
    return m2.fhex_h[i,j] == sum(m2.fhexs_h[i,p,j] for p in m2.j if p!=j) + m2.fmix_h[i,j]

m2.cons_hexs_h = pyo.Constraint(m2.i,m2.j, rule = spl_hex_h)

# Final mixer
def final_mix_c(m2,j):
    return sum(m2.fmix_c[i,j] for i in m2.i) == cold_streams['Fcp'][j]

m2.cons_mix_c = pyo.Constraint(m2.j, rule = final_mix_c)

def final_mix_c_heat(m2,j):
    return sum(m2.fmix_c[i,j]*m2.tc_out[i,j] for i in m2.i) == cold_streams['Fcp'][j]*cold_streams['Tout'][j]

m2.cons_mixh_c = pyo.Constraint(m2.j, rule = final_mix_c_heat)

def final_mix_h(m2,i):
    return sum(m2.fmix_h[i,j] for j in m2.j) == hot_streams['Fcp'][i]

m2.cons_mix_h = pyo.Constraint(m2.i, rule = final_mix_h)

def final_mix_h_heat(m2,i):
    return sum(m2.fmix_h[i,j]*m2.th_out[i,j] for j in m2.j) == hot_streams['Fcp'][i]*hot_streams['Tout'][i]

m2.cons_mixh_h = pyo.Constraint(m2.i, rule = final_mix_h_heat)

# Heat exchanger balance
def heat_hex_c(m2,i,j):
    if model.y[i,j] == 0:
        return pyo.Constraint.Skip
    else:
        return sum(model.Q[i,j,k].value for k in model.st) - m2.fhex_c[i,j]*(m2.tc_out[i,j] - m2.tc_in[i,j]) == 0 

m2.cons_heat_hex_c = pyo.Constraint(m2.i,m2.j, rule = heat_hex_c)

def heat_hex_h(m2,i,j):
    if model.y[i,j] == 0:
        return pyo.Constraint.Skip
    else:
        return sum(model.Q[i,j,k].value for k in model.st) - m2.fhex_h[i,j]*(m2.th_in[i,j] - m2.th_out[i,j]) == 0

m2.cons_heat_hex_h = pyo.Constraint(m2.i,m2.j, rule = heat_hex_h)

# Minimum temp difference
def temp1(m2,i,j):
    if model.y[i,j].value == 1:
        return m2.th_in[i,j] - m2.tc_out[i,j] >= tmapp
    else:
        return pyo.Constraint.Skip

m2.cons_temp1 = pyo.Constraint(m2.i,m2.j, rule = temp1)

def temp2(m2,i,j):
    if model.y[i,j].value == 1:
        return m2.th_out[i,j] - m2.tc_in[i,j] >= tmapp
    else:
        return pyo.Constraint.Skip
    
m2.cons_temp2 = pyo.Constraint(m2.i,m2.j, rule = temp2)

# Objective
def objective(m2):
    return  sum(sum(sum(model.Q[i,j,k].value for k in model.st)*model.y[i,j].value/((((m2.th_out[i,j] - m2.tc_in[i,j])*(m2.th_in[i,j] - m2.tc_out[i,j])\
    *((m2.th_out[i,j] - m2.tc_in[i,j])+(m2.th_in[i,j] - m2.tc_out[i,j]))/2)**0.3333)\
    *(m2.U[i,j])) for i in m2.i) for j in m2.j)

m2.OBJ = pyo.Objective(rule = objective, sense = pyo.minimize)

opt = pyo.SolverFactory('ipopt',executable='C:\\Users\\Dell\\Downloads\\ipopt-win64\\ipopt')
# opt = pyo.SolverFactory('scip', executable = 'C:\\Users\\Dell\\Downloads\\scipampl-7.0.0.win.x86_64.intel.opt.spx2.exe\\scipampl-7.0.0.win.x86_64.intel.opt.spx2')
opt.options['halt_on_ampl_error'] = 'yes'
opt.options['max_iter'] = 550
opt.options['acceptable_tol'] = 1e-05
opt.options['warm_start_init_point'] = 'yes'
# opt.options['timelimit'] = 1
results_m2 = opt.solve(m2,tee=True,symbolic_solver_labels=True)
# results_m2.write()

Ipopt 3.12.13: halt_on_ampl_error=yes
max_iter=550
acceptable_tol=1e-05
warm_start_init_point=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      292
Number of nonzeros in inequality constraint Jacobian.:       20
Number of nonzeros in Lagrangian Hessian.............:      100

Total number of variables............................:       94
                     variables with only lower bounds:       54
                variables with lower and upper b

  96  1.5539375e+01 4.11e+01 4.87e+02  -1.0 6.72e+02    -  1.50e-03 3.31e-04h  1
  97  1.6099601e+01 3.88e+01 1.23e+02  -1.0 1.04e+03    -  1.21e-03 5.66e-02h  2
  98  1.8347773e+01 3.39e+01 1.59e+02  -1.0 5.44e+02    -  1.41e-01 1.28e-01h  1
  99  1.8357319e+01 3.38e+01 1.75e+02  -1.0 1.52e+02    -  5.55e-02 1.54e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100  1.8369430e+01 3.37e+01 1.78e+02  -1.0 1.50e+02    -  4.15e-03 3.48e-03h  1
 101  1.8398253e+01 3.35e+01 1.58e+02  -1.0 1.48e+02    -  3.07e-03 5.77e-03h  1
 102  1.8435484e+01 3.34e+01 2.20e+03  -1.0 1.43e+02    -  8.63e-02 3.31e-03h  1
 103  1.8639528e+01 3.28e+01 1.43e+03  -1.0 1.38e+02    -  1.18e-02 1.72e-02h  1
 104  1.8643126e+01 3.28e+01 2.97e+03  -1.0 2.47e+02    -  2.66e-03 1.15e-03h  1
 105  1.8659144e+01 3.27e+01 6.19e+04  -1.0 4.97e+02    -  3.25e-02 3.17e-03h  1
 106  1.8659232e+01 3.26e+01 3.27e+05  -1.0 2.23e+03    -  3.34e-02 2.01e-03h  3
 107  1.8658933e+01 3.25e+01

 218  1.6719536e+01 2.31e-08 6.14e+02  -8.6 4.79e-06    -  1.00e+00 2.50e-01f  3
 219  1.6719536e+01 1.79e-08 1.34e+02  -8.6 3.66e-06    -  8.66e-01 1.00e+00H  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 220  1.6719536e+01 1.79e-08 1.84e-08  -8.6 1.20e-08    -  1.00e+00 1.00e+00h  1
 221  1.6719536e+01 1.79e-08 1.06e+03  -9.0 2.81e-07    -  4.85e-01 1.00e+00H  1
In iteration 221, 1 Slack too small, adjusting variable bound
 222  1.6719535e+01 1.75e-08 8.07e+02  -9.0 2.21e-05    -  1.56e-01 2.27e-01h  1
In iteration 222, 1 Slack too small, adjusting variable bound
 223  1.6719535e+01 1.75e-08 6.08e+02  -9.0 9.08e-06    -  8.55e-02 2.67e-01h  1
 224  1.6719535e+01 1.75e-08 3.84e+02  -9.0 4.83e-06    -  1.69e-01 1.00e+00h  1
 225  1.6719535e+01 1.75e-08 7.72e-07  -9.0 3.97e-06    -  1.00e+00 1.00e+00h  1
 226  1.6719535e+01 1.75e-08 3.07e-08  -9.0 7.45e-07    -  1.00e+00 1.00e+00h  1
 227  1.6719535e+01 1.75e-08 5.88e-09  -9.0 3.92e-08    -  1.00e+0

In [7]:
def hex_areas(m2,i,j):
    return (((m2.th_out[i,j].value - m2.tc_in[i,j].value)\
            *(m2.th_in[i,j].value - m2.tc_out[i,j].value)*((m2.th_out[i,j].value - m2.tc_in[i,j].value)\
                    +(m2.th_in[i,j].value - m2.tc_out[i,j].value))/2)**0.3333)

In [8]:
m2.area_result = pyo.Param(m2.i, m2.j, initialize=hex_areas)
m2.area_result.pprint()

area_result : Size=9, Index=area_result_index, Domain=Any, Default=None, Mutable=False
    Key    : Value
    (1, 1) : 19.994007233723313
    (1, 2) :  38.07320353138888
    (1, 3) :                0.0
    (2, 1) :  40.61305646029016
    (2, 2) :                0.0
    (2, 3) : 145.15230600044842
    (3, 1) : 119.38170170592817
    (3, 2) :                0.0
    (3, 3) :                0.0


In [25]:
# Cold streams
for i in m2.i:
    for j in m2.j:
        print(i,j,"Heat exchanger load:", sum(model.Q[i,j,k].value for k in model.st),"\nTi:",m2.tc_in[i,j].value,"\nTo:",m2.tc_out[i,j].value,"\n")

1 1 Heat exchanger load: 85.0 
Ti: 295.0000039448173 
To: 380.0000041999753 

1 2 Heat exchanger load: 195.0 
Ti: 100.0 
To: 249.99999751982529 

1 3 Heat exchanger load: 0.0 
Ti: 0.0 
To: 0.0 

2 1 Heat exchanger load: 215.0 
Ti: 160.0 
To: 320.00000359995954 

2 2 Heat exchanger load: 0.0 
Ti: 0.0 
To: 0.0 

2 3 Heat exchanger load: 225.0 
Ti: 20.0 
To: 29.999999998872163 

3 1 Heat exchanger load: 60.0 
Ti: 360.0000035398884 
To: 400.0 

3 2 Heat exchanger load: 0.0 
Ti: 0.0 
To: 0.0 

3 3 Heat exchanger load: 0.0 
Ti: 0.0 
To: 0.0 



In [26]:
# Hot streams
for i in m2.i:
    for j in m2.j:
        print(i,j,"Heat exchanger load:",sum(model.Q[i,j,k].value for k in model.st),"\nTi:",m2.th_in[i,j].value,"\nTo:",m2.th_out[i,j].value,"\n")

1 1 Heat exchanger load: 85.0 
Ti: 400.0 
To: 315.0000037448727 

1 2 Heat exchanger load: 195.0 
Ti: 315.0000065431708 
To: 120.00000394978733 

1 3 Heat exchanger load: 0.0 
Ti: 0.0 
To: 0.0 

2 1 Heat exchanger load: 215.0 
Ti: 340.0 
To: 232.50000265656823 

2 2 Heat exchanger load: 0.0 
Ti: 0.0 
To: 0.0 

2 3 Heat exchanger load: 225.0 
Ti: 232.50000324028693 
To: 120.00000152839564 

3 1 Heat exchanger load: 60.0 
Ti: 501.0 
To: 500.00000003253734 

3 2 Heat exchanger load: 0.0 
Ti: 0.0 
To: 0.0 

3 3 Heat exchanger load: 0.0 
Ti: 0.0 
To: 0.0 



In [27]:
m2.fs_c.pprint()
m2.fhexs_c.pprint()

fs_c : Size=9, Index=fs_c_index
    Key    : Lower : Value              : Upper : Fixed : Stale : Domain
    (1, 1) :     0 : 0.1562500133885401 :   1.5 : False : False : NonNegativeReals
    (1, 2) :     0 : 1.2999999999999983 :   1.3 : False : False : NonNegativeReals
    (1, 3) :     0 :                0.0 :     0 : False : False : NonNegativeReals
    (2, 1) :     0 : 1.3437499916397473 :   1.5 : False : False : NonNegativeReals
    (2, 2) :     0 :                0.0 :     0 : False : False : NonNegativeReals
    (2, 3) :     0 :               22.5 :  22.5 : False : False : NonNegativeReals
    (3, 1) :     0 :                0.0 :   1.5 : False : False : NonNegativeReals
    (3, 2) :     0 :                0.0 :     0 : False : False : NonNegativeReals
    (3, 3) :     0 :                0.0 :     0 : False : False : NonNegativeReals
fhexs_c : Size=27, Index=fhexs_c_index
    Key       : Lower : Value                 : Upper : Fixed : Stale : Domain
    (1, 1, 1) :     0 :       